In [3]:
from abc import ABC

class User(ABC):
    def __init__(self, name, email, phone, address, password, billing_info):
        self.name = name
        self.email = email
        self.phone = phone
        self.address = address
        self.password = password
        self.billing_catalog = {
            "biodegradable": 0,
            "recyclable": 0,
            "non-recyclable": 0
        }
        self.billing_info = billing_info

    def receive_warning(self, message):
        print(f"Warning to {self.name}: {message}")

    def check_total_bill(self):
        total = (self.billing_catalog['biodegradable'] * self.billing_info.biodegradable_garbage_cost +
                 self.billing_catalog['recyclable'] * self.billing_info.recyclable_garbage_cost +
                 self.billing_catalog['non-recyclable'] * self.billing_info.non_recyclable_garbage_cost)
        print(f"Bill for {self.name}: {total} Tk")

class Billing_info:
    def __init__(self):
        self.biodegradable_garbage_cost = 0
        self.recyclable_garbage_cost = 2
        self.non_recyclable_garbage_cost = 5

class Garbage:
    def __init__(self, name) -> None:
        self.name = name
        self.users = []

    def add_account_user(self, name, email, phone, address, password, billing_info):
        user = User(name, email, phone, address, password, billing_info)
        self.users.append(user)
        print(f"\t{name.capitalize()}, your registration completed. Now you can login you account")

class SourceBin:
    def __init__(self, capacity, user):
        self.capacity = capacity
        self.current_load = 0
        self.user = user

    def receive_garbadge(self, amount):
        if self.current_load + amount > self.capacity:
            print(f"Warning: Garbadge bin is full for {self.user.name}")
        else:
            self.current_load += amount
            print(f"{amount} units of garbage added. Current load: {self.current_load}/{self.capacity}")

    def send_warning(self):
        self.user.receive_warning("Reduce garbage production")

    def transfer_garbadge(self, gmp_bin, garbadge_types):
        total_garbage_to_transfer = len(garbadge_types)
        if self.current_load < total_garbage_to_transfer:
            print("No garbage left to transfer.")
            return
        for g_type in garbadge_types:
            self.current_load -= 1
            gmp_bin.allocate_garbadge(self.user, g_type)

class GMPBin(SourceBin):
    def __init__(self, capacity, user):
        super().__init__(capacity, user)
        self.biodegradable_bin = BBin()
        self.nb_bin = NBBin(10, user)

    def allocate_garbadge(self, user, garbadge):
        if garbadge == "biodegradable":
            self.biodegradable_bin.receive_garbadge(1)
            user.billing_catalog["biodegradable"] += 1
        elif garbadge == "recyclable":
            self.nb_bin.recycle_bin.receive_garbadge(1)
            user.billing_catalog["recyclable"] += 1
        elif garbadge == "non-recyclable":
            self.nb_bin.non_recycle_bin.receive_garbadge(1)
            user.billing_catalog["non-recyclable"] += 1

    def send_warning(self):
        print("Warning From GMP-Bin")

class BBin:
    def __init__(self):
        self.garbadge_count = 0

    def receive_garbadge(self, amount):
        self.garbadge_count += amount

class NBBin(SourceBin):
    def __init__(self, capacity, user):
        super().__init__(capacity, user)
        self.recycle_bin = RBin()
        self.non_recycle_bin = NRBin()

    def send_warning(self):
        print("Warning from NB-bin")

class RBin:
    def __init__(self):
        self.garbadge_count = 0

    def receive_garbadge(self, amount):
        self.garbadge_count += amount

class NRBin:
    def __init__(self):
        self.garbadge_count = 0

    def receive_garbadge(self, amount):
        self.garbadge_count += amount

Garbage_System = Garbage("Garbadge_Collection_System")
billing_info = Billing_info()

def user_menu():
    while True:
        print("\nOptions")
        print("1: Create an account")
        print("2: Login your account")
        print("3: Exit")
        option = int(input("Enter the option: "))

        if option == 1:
            name = input("Enter your name: ")
            email = input("Enter your email: ")
            phone = int(input("Enter your phone number: "))
            address = input("Enter your address: ")
            password = input("Enter your password: ")
            Garbage_System.add_account_user(name, email, phone, address, password, billing_info)
            continue
        elif option == 2:
            email = input("Enter you email: ")
            password = input("Enter you password: ")
            match = False
            client = None
            for user in Garbage_System.users:
                if user.email == email and user.password == password:
                    match = True
                    client = user
                    break
            if match == False:
                print("\tNo user found")
            else:
                source_bin = SourceBin(10, client)
                gmp_bin = GMPBin(10, user)
                print(f"\nWelcome {client.name.capitalize()}")
                while True:
                    print("\nOptions")
                    print("1: Add Garbage and Check Bill")
                    print("2: Logout")
                    user_choice = int(input("Enter your choice: "))

                    if user_choice == 1:
                        biodegradable = int(input("Enter the amount of biodegradable garbage: "))
                        recyclable = int(input("Enter the amount of recyclable garbage: "))
                        non_recyclable = int(input("Enter the amount of non-recyclable garbage: "))

                        total_garbage = ["biodegradable"] * biodegradable + ["recyclable"] * recyclable + ["non-recyclable"] * non_recyclable

                        source_bin.receive_garbadge(len(total_garbage))
                        source_bin.transfer_garbadge(gmp_bin, total_garbage)
                        client.check_total_bill()
                    elif user_choice == 2:
                        print("\nLogging out. Have a great day!")
                        break
                    else:
                        print("\tInvalid Input. Please choose a valid option.")
        elif option == 3:
            break

while True:
    print("""\t\t\t
        ************************************
        *                                  *
        *    Garbage Collection System     *
        *                                  *
        ************************************
        \n""")
    print("Options")
    print("1 : Account Holder")
    print("2 : Exit")
    option = int(input("Enter option: "))

    if option == 1:
        user_menu()
    elif option == 2:
        print("\n\tExiting the system. Have a nice day!")
        break
    else:
        print("\tInvalid Input")

			
        ************************************
        *                                  *
        *    Garbage Collection System     *
        *                                  *
        ************************************
        

Options
1 : Account Holder
2 : Exit
Enter option: 1

Options
1: Create an account
2: Login your account
3: Exit
Enter the option: 1
Enter your name: Joyant
Enter your email: joyantsheikharguptajoy@gmail.com 
Enter your phone number: 01518683509
Enter your address: Dhaka
Enter your password: 1234
	Joyant, your registration completed. Now you can login you account

Options
1: Create an account
2: Login your account
3: Exit
Enter the option: 2
Enter you email: joyantsheikharguptajoy@gmail.com 
Enter you password: 1234

Welcome Joyant

Options
1: Add Garbage and Check Bill
2: Logout
Enter your choice: 1
Enter the amount of biodegradable garbage: 2
Enter the amount of recyclable garbage: 3
Enter the amount of non-recyclable garbage: 4
9 units of garbage ad